In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

from layers.PreprocessLayer import PreprocessLayer
from utils.Utils import print_shape_dtype, pd_read_s3_parquet, upload_file 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy
import boto3
import io
import wandb

2023-04-03 03:03:18.593512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 03:03:19.213064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow versi

In [2]:
# If True, processing data from scratch
# If False, loads preprocessed data
PREPROCESS_DATA = False
TRAIN_MODEL = True
# False: use 10% of participants as validation set
# True: use all data for training -> gives better LB result
USE_VAL = True

N_ROWS = 543
N_DIMS = 2
DIM_NAMES = ['x', 'y']
SEED = 42
NUM_CLASaSES = 250

# Number of frames 
INPUT_SIZE = 32

BATCH_ALL_SIGNS_N = 4
BATCH_SIZE = 256 # Batch size during validation.
N_EPOCHS = 100
N_WARMUP_EPOCHS = 0
WD_RATIO = 0.05
MASK_VAL = 4237
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5 

In [3]:
s3_client = boto3.client(
    "s3"
)

In [4]:
AWS_S3_BUCKET = "w251-asl-data"
TRAIN_CSV_FILE = "raw-data/train.csv"

In [5]:
train_file = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=TRAIN_CSV_FILE)

In [6]:
# Read Training Data
if not PREPROCESS_DATA:
    train = pd.read_csv(train_file.get("Body")).sample(int(5e3), random_state=SEED)
else:
    train = pd.read_csv(train_file.get("Body"))

N_SAMPLES = len(train)
print(f'N_SAMPLES: {N_SAMPLES}')

N_SAMPLES: 5000


In [7]:
# Get complete file path to file
def get_file_path(path):
    return f'{AWS_S3_BUCKET}/raw-data/{path}'

train['file_path'] = train['path'].apply(get_file_path)

In [8]:
# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [9]:
display(train.head(30))
display(train.info())

,path,participant_id,sequence_id,sign,file_path,sign_ord
56533,train_landmark_files/28656/3311214787.parquet,28656,3311214787,sticky,w251-asl-data/raw-data/train_landmark_files/28...,206
63119,train_landmark_files/53618/3588192588.parquet,53618,3588192588,before,w251-asl-data/raw-data/train_landmark_files/53...,20
8760,train_landmark_files/4718/1363575346.parquet,4718,1363575346,pretty,w251-asl-data/raw-data/train_landmark_files/47...,178
93310,train_landmark_files/37779/951199059.parquet,37779,951199059,hen,w251-asl-data/raw-data/train_landmark_files/37...,114
44842,train_landmark_files/36257/283190141.parquet,36257,283190141,tomorrow,w251-asl-data/raw-data/train_landmark_files/36...,221
20993,train_landmark_files/61333/186594661.parquet,61333,186594661,up,w251-asl-data/raw-data/train_landmark_files/61...,230
89267,train_landmark_files/53618/782770724.parquet,53618,782770724,blow,w251-asl-data/raw-data/train_landmark_files/53...,25
48370,train_landmark_files/16069/2977903115.parquet,16069,2977903115,weus,w251-asl-data/raw-data/train_landmark_files/16...,236
41330,train_landmark_files/2044/269101282.parquet,2044,269101282,read,w251-asl-data/raw-data/train_landmark_files/20...,184
53090,train_landmark_files/28656/3171133897.parquet,28656,3171133897,say,w251-asl-data/raw-data/train_landmark_files/28...,191


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 56533 to 19516
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            5000 non-null   object
 1   participant_id  5000 non-null   int64 
 2   sequence_id     5000 non-null   int64 
 3   sign            5000 non-null   object
 4   file_path       5000 non-null   object
 5   sign_ord        5000 non-null   int16 
dtypes: int16(1), int64(2), object(3)
memory usage: 244.1+ KB


None

In [10]:
# Source: https://www.kaggle.com/competitions/asl-signs/overview/evaluation
ROWS_PER_FRAME = 543  # number of landmarks per frame
#w251-asl-data/raw-data/train_landmark_files/28656/3311214787.parquet

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y']
    data = pd_read_s3_parquet(pq_path[14:], AWS_S3_BUCKET, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [11]:
USE_TYPES = ['left_hand', 'pose', 'right_hand']
START_IDX = 468
LIPS_IDXS0 = np.array([
        61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
        291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
        78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
        95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
    ])
# Landmark indices in original data
LEFT_HAND_IDXS0 = np.arange(468,489)
RIGHT_HAND_IDXS0 = np.arange(522,543)
POSE_IDXS0 = np.arange(502, 512)
LANDMARK_IDXS0 = np.concatenate((LIPS_IDXS0, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, POSE_IDXS0))
HAND_IDXS0 = np.concatenate((LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0), axis=0)
N_COLS = LANDMARK_IDXS0.size
# Landmark indices in processed data
LIPS_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, LIPS_IDXS0)).squeeze()
LEFT_HAND_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, LEFT_HAND_IDXS0)).squeeze()
RIGHT_HAND_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, RIGHT_HAND_IDXS0)).squeeze()
HAND_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, HAND_IDXS0)).squeeze()
POSE_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, POSE_IDXS0)).squeeze()

print(f'# HAND_IDXS: {len(HAND_IDXS)}, N_COLS: {N_COLS}')

# HAND_IDXS: 42, N_COLS: 92


In [12]:
LIPS_START = 0
LEFT_HAND_START = LIPS_IDXS.size
RIGHT_HAND_START = LEFT_HAND_START + LEFT_HAND_IDXS.size
POSE_START = RIGHT_HAND_START + RIGHT_HAND_IDXS.size

print(f'LIPS_START: {LIPS_START}, LEFT_HAND_START: {LEFT_HAND_START}, RIGHT_HAND_START: {RIGHT_HAND_START}, POSE_START: {POSE_START}')

LIPS_START: 0, LEFT_HAND_START: 40, RIGHT_HAND_START: 61, POSE_START: 82


In [13]:
preprocess_layer = PreprocessLayer(N_ROWS, N_DIMS, HAND_IDXS0, LANDMARK_IDXS0, INPUT_SIZE)

In [14]:
"""
    face: 0:468
    left_hand: 468:489
    pose: 489:522
    right_hand: 522:544
        
"""
def get_data(file_path):
    # Load Raw Data
    data = load_relevant_data_subset(file_path)
    # Process Data Using Tensorflow
    data = preprocess_layer(data)
    
    return data

In [15]:
version = "v1"

# Get the full dataset
def get_x_y():
    # Create arrays to save data
    X = np.zeros([N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS], dtype=np.float32)
    y = np.zeros([N_SAMPLES], dtype=np.int32)
    NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, INPUT_SIZE], -1, dtype=np.float32)

    for row_idx, (file_path, sign_ord) in enumerate(tqdm(train[['file_path', 'sign_ord']].values)):
        if row_idx % 5000 == 0:
            print(f'Generated {row_idx}/{N_SAMPLES}')

        data, non_empty_frame_idxs = get_data(file_path)
        X[row_idx] = data
        y[row_idx] = sign_ord
        NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        if np.isnan(data).sum() > 0:
            print(row_idx)
            return data
    
    # Save X/y
    np.save('X.npy', X)
    np.save('y.npy', y)
    np.save('NON_EMPTY_FRAME_IDXS.npy', NON_EMPTY_FRAME_IDXS)
    
    # Put to S3
    upload_file("./X.npy", AWS_S3_BUCKET, f'processed-data/{version}/X.npy')
    upload_file("./y.npy", AWS_S3_BUCKET, f'processed-data/{version}/y.npy')
    upload_file("./NON_EMPTY_FRAME_IDXS.npy", AWS_S3_BUCKET, f'processed-data/{version}/NON_EMPTY_FRAME_IDXS.npy')
    
    return X, y, NON_EMPTY_FRAME_IDXS

In [18]:
if PREPROCESS_DATA:
    X, y, NON_EMPTY_FRAME_IDXS = get_x_y()
else:
    X = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/{version}/X.npy')
    X = np.load(io.BytesIO(X['Body'].read()))
    
    y = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/{version}/y.npy')
    y = np.load(io.BytesIO(y['Body'].read()))
    
    NON_EMPTY_FRAME_IDXS = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/{version}/NON_EMPTY_FRAME_IDXS.npy')
    NON_EMPTY_FRAME_IDXS = np.load(io.BytesIO(NON_EMPTY_FRAME_IDXS['Body'].read()))
    
print_shape_dtype([X, y, NON_EMPTY_FRAME_IDXS], ['X', 'y', 'NON_EMPTY_FRAME_IDXS'])
print(f'# NaN Values X: {np.isnan(X).sum()}')

X shape: (94477, 32, 92, 2), dtype: float32
y shape: (94477,), dtype: int32
NON_EMPTY_FRAME_IDXS shape: (94477, 32), dtype: float32
# NaN Values X: 0


In [19]:
display(pd.Series(y).value_counts().to_frame('Class Count').iloc[[0,1,2,3,4, -5,-4,-3,-2,-1]])

,Class Count
135,415
136,414
194,411
60,410
148,408
56,312
170,312
21,310
231,307
249,299
